In [2]:
import pandas as pd
import re
import transformers
from transformers import pipeline, DistilBertTokenizerFast, DistilBertForSequenceClassification
import pyttsx3
import time
import keyboard

In [3]:
from transformers import GPT2Tokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, \
                         GPT2ForSequenceClassification, set_seed, pipeline,GPT2Config

In [4]:
import load_data
from load_data import Data
import train_datasets
from train_datasets import Train_dataset

In [5]:
def generat_voice(text, pred):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)
    engine.setProperty('rate', 200)
    turn = ''
    v=0
    for i in range(len(text)):        
        turn = turn+text[i]+' '
        if pred[i]==1:
            speaker = ["A" if v == 0 else "B"]
            print(speaker, end = ' ')
            print(turn) 
            engine.say(turn)
            engine.runAndWait()
            turn = ''
            v = 1 if v == 0 else 0 
            engine.setProperty('voice', voices[v].id)

In [6]:
def display_results(snum, enum):
    
    c = 0    
    for i, t in enumerate(train_dataset.sequences[snum:enum]):
        v = 0
        print(t, end='  ')
        p_l = pipe(t)[0]['label']
        prediction.append(p_l)
        if p_l =='Continue':
            v_pred.append(v)
        else:
            c += 1
            v = 1
            v_pred.append(v)
        
        if v == train_dataset.labels[i]: 
            print([p_l], end = '')
            print(train_dataset.labels[i])
        else:
            w.append(c)
            print([p_l], end = '')
            print(train_dataset.labels[i], "<===========================")
    

In [8]:
if __name__ == '__main__':
    
    df = pd.read_json("df.json")
    t = df[14800:15000 ]
    
    train_dataset = Train_dataset(t) 
    train_dataset.test_text = t['text'].values
    train_dataset.test_label = t['label'].values
    
    train_dataset.split_token_sentences()
    train_dataset.tokenized_text_label = train_dataset.flatten_list(train_dataset.tokenized_text_label)
    train_dataset.tokenized_text = train_dataset.flatten_list(train_dataset.tokenized_text)

    train_dataset.generate_test_dataset()
    
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased', ignore_mismatched_sizes=True)
    pipe = pipeline("text-classification", './model/test_bert_distilbert_seq_token', tokenizer=tokenizer)
    
    prediction = []
    v_pred =[]
    w = []
    snum=0
    enum = 150
    print("\n\n")
    start = time.time()
    display_results(snum, enum)
    end = time.time()
    
    print("DistilBERT: time for getting results", "[",enum-snum," Tokens]", end-end)
    
    text = train_dataset.tokenized_text[snum:enum]
    v_pred = [0 if x == 'Continue' else 1 for x in prediction]    
    
    print("\n\n\nPRESS SPACE")

    while True:
        if keyboard.read_key() == "space":        
            break    
  
    generat_voice(text, v_pred)
    
    
    #GPT
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    pipe = pipeline("text-classification", './model/final_gpt2', tokenizer=tokenizer)
    
    snum=0
    enum = 150
    print("\n\n")
    start = time.time()
    display_results(snum, enum)
    end =  time.time()
    
    print("GPT2: time for getting results", "[",enum-snum," Tokens]", end-end)
    
    
    text = train_dataset.tokenized_text[snum:enum]
    v_pred = [0 if x == 'Continue' else 1 for x in prediction]    
    
    print("\n\n\nPRESS SPACE")

    while True:
        if keyboard.read_key() == "space":        
            break    
  
    generat_voice(text, v_pred)




how  ['Continue']0
how can  ['Continue']0
how can I  ['Continue']0
how can I help  ['Continue']0
how can I help you  ['Continue']0
how can I help you today?  ['Change']1
I  ['Continue']0
I would  ['Continue']0
I would like  ['Continue']0
I would like to  ['Continue']0
I would like to help  ['Continue']0
I would like to help with  ['Continue']0
I would like to help with planning  ['Continue']0
I would like to help with planning my  ['Continue']0
I would like to help with planning my summer  ['Continue']0
I would like to help with planning my summer vacation.  ['Change']1
I  ['Continue']0
I would  ['Continue']0
I would love  ['Continue']0
I would love to  ['Continue']0
I would love to help  ['Continue']0
I would love to help you  ['Continue']0
I would love to help you with  ['Continue']0
I would love to help you with your  ['Continue']0
I would love to help you with your summer  ['Continue']0
I would love to help you with your summer vacation.  ['Change']0 <===========================

I would love to help you with your summer vacation. Where are you planning to stay?  ['shift']1
I  ['Continue']0
I plan  ['Continue']0
I plan to  ['Continue']0
I plan to stay  ['Continue']0
I plan to stay in  ['Continue']0
I plan to stay in Naples,  ['Continue']0
I plan to stay in Naples, Florida  ['Continue']0
I plan to stay in Naples, Florida near  ['Continue']0
I plan to stay in Naples, Florida near the  ['Continue']0
I plan to stay in Naples, Florida near the beach.  ['shift']1
Would  ['Continue']0
Would you  ['Continue']0
Would you like  ['Continue']0
Would you like your  ['Continue']0
Would you like your hotel  ['Continue']0
Would you like your hotel to  ['Continue']0
Would you like your hotel to be  ['Continue']0
Would you like your hotel to be on  ['Continue']0
Would you like your hotel to be on the  ['Continue']0
Would you like your hotel to be on the beachfront?  ['shift']1
Yes,  ['Continue']0
Yes, I  ['Continue']0
Yes, I like  ['Continue']0
Yes, I like the  ['Continue']0
Yes